In [1]:
from copy import *

In [ ]:
class matrix:
    def __init__(self, elements:list):
        self.elements = elements
        self.size = [len(self.elements), len(self.elements[0])] if len(self.elements) > 1 else [len(self.elements[0])]
    def swap(self, i, j):
        self.elements[i], self.elements[j] = self.elements[j], self.elements[i] 
    def transpose(self):
        return matrix([[row[i] for row in self.elements] for i in range(self.size[0])])
    def determinant(self, elements, size):
        if size == 1:
            return elements[0][0]
        elif size == 2:
            return elements[0][0] * elements[1][1] - \
            elements[0][1] * elements[1][0]
        else:
            res = 0
            for col in range(size):
                sub = [[0] * (size - 1) for _ in range(size - 1)]
                for i in range(1, size):
                    subcol = 0
                    for j in range(size):
                        if j == col:
                            continue
                        sub[i - 1][subcol] = elements[i][j]
                        subcol += 1
                if col % 2 == 0:
                    sign = 1
                else:
                    sign = -1
                res += sign * elements[0][col] * self.determinant(sub, size-1)     
            return res
    def minors(self):
        result = []
        for i in range(self.size[0]):
            result.append([])
        for i in range(self.size[0]):
            for j in range(self.size[0]):
                result[i].append(0)
        for i in range(len(result)):
            for j in range(len(result)):
                c = deepcopy(self.elements)
                for k in range(len(c)):
                    del c[k][j]
                del c[i]
                m = matrix(c)
                result[i][j] = m.determinant(m.elements, m.size[0])
        return result
    def alg_complement(self):
        minor = self.minors()
        result = deepcopy(minor)
        for i in range(len(result)):
            for j in range(len(result)):
               result[i][j] = ((-1)**(i+j))*result[i][j]
        return result
    def invert(self):
        det = self.determinant(self.elements, self.size[0])
        if det == 0:
            raise ValueError("Sorry, matrix is invertible.")
        alg_T = matrix(self.alg_complement()).transpose()
        a = deepcopy(alg_T.elements)
        for i in range(alg_T.size[0]):
            for j in range(alg_T.size[0]):
                a[i][j] = a[i][j]/det
        return matrix(a)
    def multiply(self, other, vector = False):
        result = []
        for i in range(self.size[0]):
            result.append([])
            for j in range(self.size[1]):
                result[i].append(0)
        if vector == False:
            for i in range(self.size[0]): 
                for j in range(other.size[1]):
                    for k in range(other.size[0]):
                        result[i][j] += self.elements[i][k] * other.elements[k][j]
        elif vector == True:
            for i in range(self.size[0]): 
                for j in range(self.size[1]):
                    result[i][j] += self.elements[i][j] * other.elements[0][j]
        else:
            raise ValueError("vector must equal True or False.")
        return result
    def add(self, num):
        for i in range(self.size[0]):
            for j in range(self.size[1]):
                self.elements[i][j] += num
        return self.elements
    def add_matrix(self, other):
        for i in range(other.size[0]):
            for j in range(other.size[1]):
                self.add(self, other[i][j])
        return self
    def divide(self, other):
        return self.multiply(other.invert())

In [109]:
def LUP(matrixs):
    pi = [i for i in range(matrixs.size[1])]
    l, u, p = [], [], []
    for i in range(matrixs.size[0]):
        l.append([])
        u.append([])
        p.append([])
        for j in range(matrixs.size[1]):
            l[i].append(0)
            u[i].append(0)
            p[i].append(0)
    L = matrix(l)
    U = matrix(u)
    P = matrix(p)
    for k in range(matrixs.size[0]):
        max_k = -float("inf")
        for i in range(k, matrixs.size[0]):
            if max_k < abs(matrixs.elements[i][k]):
                max_k = abs(matrixs.elements[i][k])
                row_to_swap = i
        if row_to_swap != k:
            pi[k], pi[row_to_swap] = pi[row_to_swap], pi[k]
            matrixs.swap(k, row_to_swap)
        for i in range(k + 1, matrixs.size[0]):
            matrixs.elements[i][k] /= matrixs.elements[k][k]
            for j in range(k + 1, matrixs.size[1]):
                matrixs.elements[i][j] -= matrixs.elements[i][k] * matrixs.elements[k][j]
    for row in range(matrixs.size[0]):
        for col in range(matrixs.size[1]):
            if col == pi[row]:
                P.elements[row][col] = 1
            if row <= col:
                if row == col:
                    L.elements[row][col] = 1
                U.elements[row][col] = matrixs.elements[row][col]
            else:
                L.elements[row][col] = matrixs.elements[row][col]
    return L, U, P

In [293]:
def solve(matrixs, b):
    l, u, p = LUP(matrixs)
    p1 = deepcopy(p)
    b1 = matrix([deepcopy(b)])
    pb = []
    for i in range(p1.size[0]):
        for j in range(p1.size[1]):
            if p1.elements[i][j] == 1:
                pb.append([b1.elements[0][j]])
    y1, x1 = [], []
    for i in range(matrixs.size[0]):
        y1.append([0])
        x1.append([0])
    y = matrix(y1)
    x = matrix(x1)
    for k in range(matrixs.size[0]):
        sum = 0
        for i in range(k):
            sum += l.elements[k][i] * y.elements[i][0]
        y.elements[k] = [pb[k][0] - sum]
    for k in range(matrixs.size[0] - 1, -1, -1):
        sum = 0
        for i in range(k + 1, matrixs.size[0]):
            sum += u.elements[k][i] * x.elements[i][0]
        x.elements[k] = [(y.elements[k][0] - sum) / u.elements[k][k]]
    return x

In [19]:
A = matrix([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
n = matrix([[7, 8, 0], [1, 4, 2], [8, 6, 5]])

In [21]:
A.multiply(n)

[[33, 34, 19], [81, 88, 40], [129, 142, 61]]

In [23]:
A.determinant(A.elements, len(A.elements))

0

In [25]:
A.minors()

[[-3, -6, -3], [-6, -12, -6], [-3, -6, -3]]

In [27]:
A.alg_complement()

[[-3, 6, -3], [6, -12, 6], [-3, 6, -3]]

In [29]:
n.determinant(n.elements, n.size[0])

144

In [31]:
n.invert().elements

[[0.05555555555555555, -0.2777777777777778, 0.1111111111111111],
 [0.0763888888888889, 0.24305555555555555, -0.09722222222222222],
 [-0.18055555555555555, 0.1527777777777778, 0.1388888888888889]]

In [57]:
A = matrix([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
k = matrix([[8, 9, 3]])
A.multiply(k, vector = True)

[[8, 18, 9], [32, 45, 18], [56, 72, 27]]

In [33]:
A.divide(n)

[[-0.33333333333333326, 0.6666666666666667, 0.33333333333333337],
 [-0.4791666666666665, 1.0208333333333333, 0.7916666666666667],
 [-0.625, 1.3749999999999998, 1.25]]

In [ ]:
solve_for = matrix([[1, 1, 0, 3, 0, 0],
                    [7, 4, 8, 0, 9, 0],
                    [0, 5, 4, 0, 0, 9],
                    [3, 0, 0, 4, 8, 0],
                    [0, 6, 0, 1, 4, 5],
                    [0, 0, 5, 0, 7, 4]])
b = [1, 5, 0, 2, 4, 1]

In [ ]:
solve(solve_for, b).elements

[[-0.4793489318413024],
 [0.9122075279755851],
 [0.2111902339776197],
 [0.18904713462190598],
 [0.33523228212953543],
 [-0.6006442861987117]]

In [299]:
import numpy as np
a = [[1, 1, 0, 3, 0, 0],
    [7, 4, 8, 0, 9, 0],
    [0, 5, 4, 0, 0, 9],
    [3, 0, 0, 4, 8, 0],
    [0, 6, 0, 1, 4, 5],
    [0, 0, 5, 0, 7, 4]]
x = np.linalg.solve(a, b)

In [301]:
x

array([-0.47934893,  0.91220753,  0.21119023,  0.18904713,  0.33523228,
       -0.60064429])